In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt

# Define the ticker symbol for the stock you want to retrieve options data for (e.g., Apple)
ticker_symbol = 'AAPL'

# Download stock data using yfinance
ticker = yf.Ticker(ticker_symbol)

# Fetch the most recent stock price (current price of the underlying asset)
current_price = ticker.history(period="1d")['Close'][-1]

# Fetch options expiration dates
expiration_dates = ticker.options

# Pick an expiration date for which you want to download option data
# For this example, let's choose the nearest expiration date
selected_expiration_date = expiration_dates[0]

# Fetch option chain for the selected expiration date
option_chain = ticker.option_chain(selected_expiration_date)

# Get calls and puts
calls = option_chain.calls
puts = option_chain.puts

# Combine calls and puts into one DataFrame
options_data = pd.concat([calls, puts], keys=['Call', 'Put']).reset_index(level=0)
options_data.rename(columns={'level_0': 'Option_Type'}, inplace=True)

# Add the 'current_price', 'time_to_maturity', 'interest_rate', and 'volatility' columns
# Assumptions for this example
interest_rate = 0.05  # 5% risk-free rate (assumed, can be fetched or updated)
today = dt.datetime.today()

# Calculating the 'time_to_maturity' in years
options_data['expiration'] = pd.to_datetime(selected_expiration_date)
options_data['time_to_maturity'] = (options_data['expiration'] - today).dt.days / 365.0

# Assuming historical volatility for now, could fetch implied volatility from options chain (if available)
options_data['volatility'] = options_data['impliedVolatility']

# Adding the current price and interest rate columns
options_data['current_price'] = current_price
options_data['interest_rate'] = interest_rate

# Display selected columns (current_price, strike_price, time_to_maturity, volatility, interest_rate)
options_data_display = options_data[['Option_Type', 'strike', 'lastPrice', 'current_price', 'time_to_maturity', 'volatility', 'interest_rate']]

# Display the dataframe
print(options_data_display.head())

# Save the final DataFrame to CSV for further use
options_data_display.to_csv(f"{ticker_symbol}_options_data_with_features.csv", index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15348\3307671940.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = ticker.history(period="1d")['Close'][-1]


  Option_Type  strike  lastPrice  current_price  time_to_maturity  volatility  \
0        Call   100.0     124.05     228.813599           0.00274    3.886719   
1        Call   120.0     105.50     228.813599           0.00274    3.148440   
2        Call   135.0      90.24     228.813599           0.00274    2.707034   
3        Call   140.0      88.20     228.813599           0.00274    2.500004   
4        Call   145.0      80.25     228.813599           0.00274    2.300785   

   interest_rate  
0           0.05  
1           0.05  
2           0.05  
3           0.05  
4           0.05  


In [2]:
import yfinance as yf
import pandas as pd
import datetime as dt

# Define the ticker symbol for the stock you want to retrieve options data for (e.g., Apple)
ticker_symbol = 'AAPL'

# Download stock data using yfinance
ticker = yf.Ticker(ticker_symbol)

# Get the historical stock prices for the last 10 years
historical_data = ticker.history(period="10y")

# Get the current date and calculate the earliest date for options
current_date = dt.datetime.now()
earliest_date = current_date - dt.timedelta(days=365 * 10)

# Fetch options expiration dates within the last 10 years
expiration_dates = ticker.options
options_data = []

# Loop through each expiration date to fetch option chain data
for expiration_date in expiration_dates:
    expiration_dt = pd.to_datetime(expiration_date)
    
    # Only fetch data if the expiration date is within the last 10 years
    if expiration_dt > earliest_date:
        option_chain = ticker.option_chain(expiration_date)
        
        # Get calls and puts
        calls = option_chain.calls
        puts = option_chain.puts
        
        # Combine calls and puts into one DataFrame and add expiration date
        calls['Option_Type'] = 'Call'
        puts['Option_Type'] = 'Put'
        calls['expiration'] = expiration_dt
        puts['expiration'] = expiration_dt
        
        # Append to the options_data list
        options_data.append(calls)
        options_data.append(puts)

# Combine all options data into a single DataFrame
options_data_combined = pd.concat(options_data, ignore_index=True)

# Filter relevant columns
options_data_display = options_data_combined[
    ['Option_Type', 'contractSymbol', 'strike', 'lastPrice', 'bid', 'ask', 'change', 
     'percentChange', 'volume', 'openInterest', 'impliedVolatility', 'expiration']
]

# Display the first few rows of options data
print(options_data_display.head())

# Save the options data to CSV for further analysis
options_data_display.to_csv(f"{ticker_symbol}_options_data_10_years.csv", index=False)

# Display historical stock price data
print(historical_data.head())

# Save historical stock price data to CSV
historical_data.to_csv(f"{ticker_symbol}_historical_stock_data.csv")


  Option_Type       contractSymbol  strike  lastPrice     bid     ask  \
0        Call  AAPL241011C00100000   100.0     124.05  128.55  129.40   
1        Call  AAPL241011C00120000   120.0     105.50  108.60  109.40   
2        Call  AAPL241011C00135000   135.0      90.24   93.70   94.40   
3        Call  AAPL241011C00140000   140.0      88.20   88.65   89.40   
4        Call  AAPL241011C00145000   145.0      80.25   83.55   84.45   

     change  percentChange  volume  openInterest  impliedVolatility expiration  
0  0.000000       0.000000     2.0             1           3.992188 2024-10-11  
1  0.000000       0.000000     1.0             1           3.222658 2024-10-11  
2  0.000000       0.000000     NaN             4           2.765628 2024-10-11  
3  5.089996       6.124409     1.0            15           2.558597 2024-10-11  
4  0.000000       0.000000    69.0             5           2.359379 2024-10-11  
                                Open       High        Low      Close  \
Da